In [1]:
import os

# List the current directory
#print(os.listdir(.))

In [ ]:
# Ask the user to select a digit in the range 0 to 8
k = int(input("Please select a digit in the range 0 to 8: "))

# Check if the digit is within the valid range
if 0 <= k <= 8:
    print(f"will be running in k{k}")
else:
    print("Invalid selection. Please select a digit between 0 and 8.")
    exit(1)

from pathlib import Path

wd = Path(f"/home/rfriedma/src/k{k}/ceph/build")
os.chdir(wd)
%env CEPH_JTEST_ROOT=/home/rfriedma/src/k{k}/ceph/build
!echo $CEPH_JTEST_ROOT > /tmp/jpath


In [ ]:
!ls -l
!bash -c MGR=0 ls -l


In [ ]:
%%bash
scrtch=to_"`date +%d_%H%M`"
echo $scrtch

MDS=0 MGR=1 OSD=5 MON=1 ../src/vstart.sh -n  --without-dashboard --msgr2 -X --memstore -o "memstore_device_bytes=68435456" -o "osd_op_queue=wpq"
sleep 2
bin/ceph -s

bin/ceph tell osd.* config set debug_osd 20/20

bin/ceph config set global osd_pool_default_pg_autoscale_mode off
sleep 2

# disable rescheduling of the queue due to 'no-scrub' flags
#bin/ceph tell osd.* config set osd_scrub_backoff_ratio 0.9999

# initial set of global scrub scheduling parameters
bin/ceph tell osd.* config set osd_scrub_interval_randomize_ratio 0.1
bin/ceph tell osd.* config set osd_deep_scrub_randomize_ratio 0
bin/ceph tell osd.* config set osd_scrub_min_interval 10
bin/ceph tell osd.* config set osd_scrub_max_interval 2000
bin/ceph tell osd.* config set osd_deep_scrub_interval 600


#PL1 is of size 3

bin/ceph osd pool create pl1 8 8
#bin/ceph osd pool autoscale-status
sleep 1
pl1_num=`bin/ceph osd pool stats pl1 | sed -n -r -e 's/.*id[^0-9]*([0-9]+)$/\1/p'`
echo $pl1_num
bin/ceph osd pool set pl1 size 3
bin/ceph osd pool set pl1 min_size 3
bin/ceph osd pool set pl1 pg_autoscale_mode off
bin/ceph osd pool stats
bin/ceph osd pool set pl1 noscrub 1
bin/ceph osd pool set pl1 nodeep-scrub 1
sleep 2

bin/rados bench -p pl1 -t 1 1 write -b 4096 --max-objects 8  --no-cleanup; 
bin/rados bench -p pl1 1 write -b 4096 --max-objects 128 --show-time --no-cleanup --run-name eeeee

#PL2

bin/ceph osd pool create pl2 16 16
bin/ceph osd pool set pl2 size 3
bin/ceph osd pool set pl2 min_size 3
bin/ceph osd pool set pl2 pg_autoscale_mode off
bin/ceph osd pool stats
bin/ceph osd pool set pl2 noscrub 1
bin/ceph osd pool set pl2 nodeep-scrub 1
sleep 2

bin/rados bench -p pl2 -t 1 1 write -b 4096 --max-objects 64  --no-cleanup; 
bin/rados bench -p pl2 1 write -b 4096 --max-objects 128 --show-time --no-cleanup --run-name eeeee
sleep 1
bin/ceph tell osd.* config set debug_osd 20/20

# the set of PGs
bin/ceph pg dump
bin/ceph pg dump pgs_brief
bin/ceph pg dump pgs_brief -f=json-pretty

In [ ]:
%%bash
echo '-------->' $CEPH_JTEST_ROOT
jp=$CEPH_JTEST_ROOT
if [ -z $CEPH_JTEST_ROOT ]; then
    echo "CEPH_JTEST_ROOT is not set"
    [[ -f /tmp/jpath ]] && jp=`cat /tmp/jpath` || jp='.'
    echo '-------->' $jp
    %env CEPH_JTEST_ROOT=$jp
fi
cd $jp


In [ ]:
%%bash

file_path="/tmp/jpath"
if [ -f "$file_path" ]; then
        file_contents=$(cat "$file_path")
        echo "File contents read into variable."
else
        echo "File does not exist."
fi

In [ ]:
%%bash
nosd=5
for ((i=0;i<=$nosd;i++)); do
  echo "Query map for OSD $i"
  bin/ceph pg $pl1_num.$i query | jq '.scrubber' 
done


In [ ]:
%%bash

#cd $CEPH_JTEST_ROOT

bin/ceph tell osd.0 dump_scrubs --format=json-pretty > /tmp/ds_00_b4params.json
bin/ceph tell osd.1 dump_scrubs --format=json-pretty > /tmp/ds_01_b4params.json
bin/ceph tell osd.2 dump_scrubs --format=json-pretty > /tmp/ds_02_b4params.json


# set the scheduling parameters

bin/ceph osd pool set pl1 scrub_min_interval 1
bin/ceph osd pool set pl1 scrub_max_interval 2
bin/ceph osd pool set pl1 deep_scrub_interval 1000

#bin/ceph osd pool set pl2 scrub_min_interval 2
#bin/ceph osd pool set pl2 scrub_max_interval 4
#bin/ceph osd pool set pl2 deep_scrub_interval 10

sleep 3

bin/ceph tell osd.0 dump_scrubs --format=json-pretty > /tmp/ds_00.json
bin/ceph tell osd.1 dump_scrubs --format=json-pretty > /tmp/ds_01.json
bin/ceph tell osd.2 dump_scrubs --format=json-pretty > /tmp/ds_02.json
#bin/ceph tell osd.3 dump_scrubs --format=json-pretty > /tmp/ds_02.json





In [ ]:
%%bash

#%cd $CEPH_JTEST_ROOT

# common scrub configs
bin/ceph tell osd.* config set osd_blocked_scrub_grace_period 20
bin/ceph tell osd.* config set osd_stats_update_period_scrubbing 2
bin/ceph tell osd.* config set osd_stats_update_period_not_scrubbing 3
#bin/ceph tell osd.* config set osd_scrub_backoff_ratio 0.9999
#bin/ceph tell osd.* config set osd_scrub_interval_randomize_ratio 0.1
#bin/ceph tell osd.* config set osd_deep_scrub_randomize_ratio 0

#bin/ceph tell mgr.$(bin/ceph mgr services | jq -r .mgr) config set mgr_stats_period 2


In [ ]:
%%bash

# list the scrub queue
scrtch=to_"`date +'%H%M%S'`"
echo $scrtch
bin/ceph tell osd.0 dump_scrubs --format=json-pretty
bin/ceph tell osd.1 dump_scrubs --format=json-pretty
bin/ceph tell osd.2 dump_scrubs --format=json-pretty
bin/ceph tell osd.0 dump_scrubs --format=json-pretty > /tmp/ds_0$scrtch.json
bin/ceph tell osd.1 dump_scrubs --format=json-pretty > /tmp/ds_1$scrtch.json
bin/ceph tell osd.2 dump_scrubs --format=json-pretty > /tmp/ds_2$scrtch.json


In [ ]:
%%bash
# set scrub parameters to guarantee slow scrub
bin/ceph tell osd.* config set osd_scrub_sleep "3.0"
bin/ceph tell osd.* config set osd_max_scrubs 1
bin/ceph tell osd.* config set osd_scrub_chunk_max 5
bin/ceph tell osd.* config set osd_shallow_scrub_chunk_max 5


In [ ]:
%%bash

# set higher urgency to one of the PGs
bin/ceph tell $pl1_num.7 scrub
bin/ceph tell $pl1_num.6 schedule-deep-scrub
sleep 1
bin/ceph pg dump pgs


In [ ]:
%%bash

scrtch=to_"`date +'%H%M%S'`"
echo $scrtch

# list the scrub queue
bin/ceph tell osd.0 dump_scrubs --format=json-pretty > /tmp/ds_0$scrtch.json
bin/ceph tell osd.1 dump_scrubs --format=json-pretty > /tmp/ds_1$scrtch.json
bin/ceph tell osd.2 dump_scrubs --format=json-pretty > /tmp/ds_2$scrtch.json
bin/ceph tell osd.0 dump_scrubs --format=json-pretty
bin/ceph tell osd.1 dump_scrubs --format=json-pretty
bin/ceph tell osd.2 dump_scrubs --format=json-pretty


In [ ]:
%%bash

# the idea now is to build on the previous attempt, and:
# 1 - create a dictionary of refs to per-pg dictionary of the data (or - if too complex - the data as a list)
# 2 - return multiple objects:
# 1) a full dict as above
# 2) PGs to primary
# 3) PGs to acting set
# 4) pool to PGs
# 5) PG to pool

function build_pg_dicts {
  local infile=$1
  local -n pg_primary_dict=$2
  local -n pg_acting_dict=$3
  local -n pg_pool_dict=$4

  local extr_dbg=2 # note: 3 and above leave some temp files around

  #turn off '-x' (but remember previous state)
  local saved_echo_flag=${-//[^x]/}
  set +x

  # if the infile name is '-', fetch the dump directly from the ceph cluster
  if [[ $infile == "-" ]]; then
    local -r ceph_cmd="bin/ceph pg dump pgs_brief -f=json-pretty"
    local -r ceph_cmd_out=$(eval $ceph_cmd)
    local -r ceph_cmd_rc=$?
    if [[ $ceph_cmd_rc -ne 0 ]]; then
      echo "Error: the command '$ceph_cmd' failed with return code $ceph_cmd_rc"
      #return $ceph_cmd_rc
    fi
    echo "$ceph_cmd_out" > /tmp/e2
    infile='/tmp/e2'
  fi

  l0=`jq '[.pg_stats | group_by(.pg_stats)[0] | map({pgid: .pgid, pool: (.pgid | split(".")[0]), acting: .acting, acting_primary: .acting_primary})] | .[]' $infile `
  (( extr_dbg >= 2 )) && echo "L0: $l0"

  mapfile -t l1 < <(echo "$l0" | jq -c '.[]')
  (( extr_dbg >= 2 )) && echo "L1: ${#l1[@]}"

  for item in "${l1[@]}"; do
    pgid=$(echo "$item" | jq -r '.pgid')
    acting=$(echo "$item" | jq -r '.acting | @sh')
    pg_acting_dict["$pgid"]=$acting
    acting_primary=$(echo "$item" | jq -r '.acting_primary')
    pg_primary_dict["$pgid"]=$acting_primary
    pool=$(echo "$item" | jq -r '.pool')
    pg_pool_dict["$pgid"]=$pool
    #pool_dict["$pgid"]="acting=($acting) acting_primary=$acting_primary pool=$pool"
  done
}

declare -A pg_pr
declare -A pg_ac
declare -A pg_po
build_pg_dicts "-" pg_pr pg_ac pg_po

echo "PGs to primary:"
for pg in "${!pg_pr[@]}"; do
  echo "Got: $pg: ${pg_pr[$pg]} ( ${pg_ac[$pg]} ) ${pg_po[$pg]}"
done



# a function that counts the number of common active-set elements between two PGs
# 1 - the first PG
# 2 - the second PG
# 3 - the dictionary of active sets
function count_common_active {
  local pg1=$1
  local pg2=$2
  local -n pg_acting_dict=$3
  local -n res=$4

  local -a a1=(${pg_acting_dict[$pg1]})
  local -a a2=(${pg_acting_dict[$pg2]})

  local -i cnt=0
  for i in "${a1[@]}"; do
    for j in "${a2[@]}"; do
      if [[ $i -eq $j ]]; then
        cnt=$((cnt+1))
      fi
    done
  done

  res=$cnt
}

# a function that returns an array of the common active-set elements between two PGs
# 1 - the first PG
# 2 - the second PG
# 3 - the dictionary of active sets
function get_common_active {
  local pg1=$1
  local pg2=$2
  local -n actng=$3
  local -n res=$4

  local -a a1=(${actng[$pg1]})
  local -a a2=(${actng[$pg2]})

  local -a common=()
  for i in "${a1[@]}"; do
    for j in "${a2[@]}"; do
      if [[ $i -eq $j ]]; then
        common+=($i)
      fi
    done
  done

  res=(${common[@]})
}


# given a PG, find another one with a disjoint active set
# 1 - the PG
# 2 - the dictionary of active sets
# 3 - [out] - the PG with a disjoint active set
function find_disjoint_pg {
  local pg=$1
  local -n ac_dict=$2
  local -n res=$3

  for cand in "${!ac_dict[@]}"; do
    if [[ $cand != $pg ]]; then
      local -i common=0
      count_common_active $pg $cand ac_dict common
      if [[ $common -eq 0 ]]; then
        res=$cand
        return
      fi
    fi
  done
}

echo "Testing the find_disjoint_pg function"
rs4=""
find_disjoint_pg "2.5" pg_ac rs4
echo "The result: $rs4"


echo "done"

In [ ]:
raise SystemExit("Stop here")

# Termination


In [ ]:
%%bash
echo '-------->' $CEPH_JTEST_ROOT
cd $CEPH_JTEST_ROOT
if [ -z $CEPH_JTEST_ROOT ]; then
    echo "CEPH_JTEST_ROOT is not set"
    [[ -f /tmp/jpath ]] && jp=`cat /tmp/jpath` || jp='.'
    echo '-------->' $jp
    cd $jp
    %env CEPH_JTEST_ROOT=$jp
fi

pwd

../src/stop.sh
sleep 4
../src/stop.sh
